## Read input

In [209]:
import re

total = 0

with open('data/05.txt', 'r') as file:
    seeds = list(map(int, re.findall(r'\d+', file.readline())))
    maps = {}
    key = ''

    for line in file.readlines():
        if 'map' in line:
            key = re.split(' |-', line)[2]
            maps[key] = []
            continue

        if line.strip() == '':
            continue

        maps[key].append(list(map(int, re.findall(r'\d+', line))))

## Part 1

In [210]:
def get_from_map(source, key):
    value = False

    for destination, map_source, map_length in maps[key]:
        if source in range(map_source, map_source + map_length):
            if not value:
                value = destination + source - map_source

    return value if value else source

locations = []

for seed in seeds:
    value = seed

    for key in list(maps.keys()):
        value = get_from_map(value, key)

    locations.append(value)

min(locations)

240320250

## Part 2

In [211]:
def merge_intervals(intervals):
    intervals.sort(key=lambda x: x[0])
    index = 0

    for i in range(1, len(intervals)):
        if (intervals[index][1] + 1 >= intervals[i][0]):
            intervals[index][1] = max(intervals[index][1], intervals[i][1])
        else:
            index = index + 1
            intervals[index] = intervals[i]
 
    return intervals[:index + 1]

def get_from_range(start, length, key):
    matched = []
    end = start + length - 1
    
    for destination, map_start, map_length in maps[key]:
        map_end = map_start + map_length - 1

        if max(start, map_start) <= min(end, map_end):
            new_start = max(start, map_start)
            new_end = min(end, map_end)
            new_length = (new_end - new_start) + 1

            yield [destination + new_start - map_start, new_length]

            matched.append([new_start, new_end])

    matched = merge_intervals(matched)

    if (len(matched) == 0):
        yield [start, length]

    for map_start, map_end in matched:
        if start < map_start:
            new_end = min(end, map_start - 1)
            new_length = (new_end - start) + 1
            yield [start, new_length]

        if end > map_end:
            new_start = max(start, map_end + 1)
            new_length = (end - new_start) + 1
            yield [new_start, new_length]

locations = []
keys = list(maps.keys())
pairs = [seeds[i:i + 2] for i in range(0, len(seeds), 2)]

for start, length in pairs:
    queue = [[start, length, 'soil']]

    while queue:
        start, end, key = queue.pop(0)

        for result in get_from_range(start, end, key):
            next_key = keys.index(key) + 1

            if next_key >= len(keys):
                locations.append(result[0])
            else:
                queue.append([*result, keys[next_key]])

min(locations)


28580589